# Model with Feedback

Example testing a model with both predict and feedback in python. This could be used as a basis for a reinforcement learning model deployment.
 

# REST

In [ ]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-feedback-rest:0.1

In [ ]:
!docker run --name "model-with-feedback" -d --rm -p 5000:5000 model-with-feedback-rest:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

## Test feedback

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --endpoint send-feedback

In [ ]:
!docker rm model-with-feedback --force

# gRPC

In [ ]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-feedback-grpc:0.1

In [ ]:
!docker run --name "model-with-feedback" -d --rm -p 5000:5000 model-with-feedback-grpc:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

## Test predict

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

## Test feedback

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --endpoint send-feedback --grpc

In [ ]:
!docker rm model-with-feedback --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need Minikube version 0.25.2**

In [8]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

Starting local Kubernetes v1.9.4 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [9]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [10]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [11]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [12]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Oct 13 11:28:53 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s

==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Sat Oct 13 11:28:54 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/ClusterRole
NAME        AGE
seldon-crd  0s

==> v1/ClusterRoleBindin

# REST

In [13]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-feedback-rest:0.1 --loglevel 5

I1013 11:29:08.699259   21892 build.go:50] Running S2I version "v1.1.9a"
I1013 11:29:08.699394   21892 util.go:57] Getting docker credentials for seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1013 11:29:08.699416   21892 util.go:73] Using  credentials for pulling seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1013 11:29:08.729338   21892 docker.go:519] error inspecting image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: Error: No such image: seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1013 11:29:08.729358   21892 docker.go:506] Image "seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT" not available locally, pulling ...
I1013 11:29:10.193024   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:10.193042   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:10.193051   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:10.193059   21892 docker.go:578] pullin

I1013 11:29:12.549005   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  9.678MB/50.06MB
I1013 11:29:12.645817   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  8.711MB/45.31MB
I1013 11:29:12.659347   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  11.72MB/50.06MB
I1013 11:29:12.714873   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  2.145MB/213.2MB
I1013 11:29:12.759366   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  13.77MB/50.06MB
I1013 11:29:12.817788   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  2.677MB/213.2MB
I1013 11:29:12.824387   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:   9.17MB/45.31MB
I1013 11:29:12.862147   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  15.81MB/50.06MB
I1013 11:29:12.9

I1013 11:29:16.186382   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  13.77MB/45.31MB
I1013 11:29:16.189193   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:   59.5MB/213.2MB
I1013 11:29:16.293343   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  62.16MB/213.2MB
I1013 11:29:16.398154   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  64.83MB/213.2MB
I1013 11:29:16.456559   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  46.79MB/50.06MB
I1013 11:29:16.503665   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  67.53MB/213.2MB
I1013 11:29:16.608726   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  70.22MB/213.2MB
I1013 11:29:16.713105   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:   72.9MB/213.2MB
I1013 11:29:16.8

I1013 11:29:21.215853   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  16.52MB/45.31MB
I1013 11:29:21.220274   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  5.156MB/5.745MB
I1013 11:29:21.260535   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  5.745MB/5.745MB
I1013 11:29:21.260655   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:21.285489   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  187.8MB/213.2MB
I1013 11:29:21.393302   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  190.5MB/213.2MB
I1013 11:29:21.510676   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  193.7MB/213.2MB
I1013 11:29:21.630573   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  196.9MB/213.2MB
I1013 11:29:21.730840   21892 do

I1013 11:29:25.043800   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  6.679MB/38.45MB
I1013 11:29:25.077202   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  35.83MB/45.31MB
I1013 11:29:25.116991   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:     135B/135B
I1013 11:29:25.117391   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:25.117425   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:25.156316   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  8.645MB/38.45MB
I1013 11:29:25.196498   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:   37.2MB/45.31MB
I1013 11:29:25.280294   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  10.61MB/38.45MB
I1013 11:29:25.311265   21892 docker.go:578] pullin

...

I1013 11:29:31.035092   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  39.14MB/41.48MB
I1013 11:29:31.099013   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  8.389MB/50.06MB
I1013 11:29:31.208740   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  11.01MB/50.06MB
I1013 11:29:31.257113   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  39.56MB/41.48MB
I1013 11:29:31.322078   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  13.63MB/50.06MB
I1013 11:29:31.440454   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  16.25MB/50.06MB
I1013 11:29:31.568258   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  18.35MB/50.06MB
I1013 11:29:31.675662   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  19.92MB/50.06MB
I1013 11:29:31.7

I1013 11:29:38.525285   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  82.44MB/213.2MB
I1013 11:29:38.632107   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  85.79MB/213.2MB
I1013 11:29:38.743181   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  89.13MB/213.2MB
I1013 11:29:38.859429   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  91.91MB/213.2MB
I1013 11:29:38.962587   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:   94.7MB/213.2MB
I1013 11:29:39.081219   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  97.48MB/213.2MB
I1013 11:29:39.185847   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  100.8MB/213.2MB
I1013 11:29:39.291732   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  103.6MB/213.2MB
I1013 11:29:39.3

I1013 11:29:45.811261   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  1.245MB/1.788MB
I1013 11:29:45.840128   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  1.788MB/1.788MB
I1013 11:29:45.842266   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  1.788MB/1.788MB
I1013 11:29:45.868300   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:45.904949   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  131.1kB/12.37MB
I1013 11:29:46.005945   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  3.015MB/12.37MB
I1013 11:29:46.108468   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  6.554MB/12.37MB
I1013 11:29:46.210401   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  10.22MB/12.37MB
I1013 11:29:46.273605   21892 do

I1013 11:29:51.508783   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT:  1.573kB/1.573kB
I1013 11:29:51.544862   21892 docker.go:578] pulling image seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT: 
I1013 11:29:51.564452   21892 build.go:150] 
Builder Image:			seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
Source:				.
Output Image Tag:		model-with-feedback-rest:0.1
Environment:			MODEL_NAME=ModelWithFeedback,API_TYPE=REST,SERVICE_TYPE=MODEL,PERSISTENCE=0
Environment File:		environment_rest
Labels:				
Incremental Build:		disabled
Remove Old Build:		disabled
Builder Pull Policy:		if-not-present
Previous Image Pull Policy:	if-not-present
Quiet:				disabled
Layered Build:			disabled
Docker Endpoint:		tcp://192.168.39.88:2376
Docker Pull Config:		/home/clive/.docker/config.json
Docker Pull User:		cliveseldon

I1013 11:29:51.565657   21892 docker.go:510] Using locally available image "seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT"
I1013 11:29:51.570531   21892

I1013 11:29:51.813097   21892 docker.go:1055] Waiting for container "0469954fa0a4ac2dbb4cdcb42beea6828dc629dc4334942bfdd793af17007b34" to stop ...
I1013 11:29:51.867706   21892 docker.go:1080] Invoking PostExecute function
I1013 11:29:51.867719   21892 postexecutorstep.go:67] Skipping step: store previous image
I1013 11:29:51.867724   21892 postexecutorstep.go:116] Executing step: commit image
I1013 11:29:51.869206   21892 postexecutorstep.go:521] Checking for new Labels to apply... 
I1013 11:29:51.869218   21892 postexecutorstep.go:529] Creating the download path '/tmp/s2i357918663/metadata'
I1013 11:29:51.869271   21892 postexecutorstep.go:463] Downloading file "/tmp/.s2i/image_metadata.json"
I1013 11:29:51.911999   21892 postexecutorstep.go:537] unable to download and extract 'image_metadata.json' ... continuing
I1013 11:29:51.914973   21892 docker.go:1114] Committing container with dockerOpts: {Reference:model-with-feedback-rest:0.1 Comment: Author: Changes:[] Pause:false Config:0x

In [14]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [15]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-svc-orch replicas:1 replicasAvailable:1] map[name:mymodel-mymodel-complex-model-0 replicas:1 replicasAvailable:1]] state:Available]

In [16]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [17]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[4.165728345860066, 3.421125351987684, 5.347988225860308, 0.34862197994993105]]}}
Getting token from http://192.168.39.88:30552/oauth/token
{"access_token":"83061770-7057-4e82-aa6d-ea310e9749b3","token_type":"bearer","expires_in":43199,"scope":"read write"}
RECEIVED RESPONSE:
{'meta': {'puid': '3h96s6vgngcnokui05uu99on8t', 'tags': {}, 'routing': {}, 'requestPath': {'complex-model': 'model-with-feedback-rest:0.1'}}, 'data': {'names': ['t:0', 't:1', 't:2', 't:3'], 'ndarray': [[4.165728345860066, 3.421125351987684, 5.347988225860308, 0.34862197994993105]]}}



## Test feedback

In [18]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --endpoint send-feedback

----------------------------------------
SENDING NEW REQUEST:
{'request': {'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[6.231069700941248, 4.672819188325931, 9.82963767564628, 2.9349660518483818]]}}, 'response': {'meta': {}, 'data': {'names': ['class1', 'class2', 'class3'], 'ndarray': [[0.9325544700393164, 0.3367054849207918, 0.7675130586137497]]}}, 'reward': 1.0}
Getting token from http://192.168.39.88:30552/oauth/token
{"access_token":"83061770-7057-4e82-aa6d-ea310e9749b3","token_type":"bearer","expires_in":43196,"scope":"read write"}
<Response [200]>


In [19]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


# gRPC

In [35]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-feedback-grpc:0.1

---> Installing application source...
Build completed successfully


In [36]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [37]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-svc-orch replicas:1 replicasAvailable:0] map[replicasAvailable:1 name:mymodel-mymodel-complex-model-0 replicas:1]]]

In [38]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [39]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 4.676817651188339
        }
        values {
          number_value: 3.364202436251853
        }
        values {
          number_value: 9.38636971737447
        }
        values {
          number_value: 0.873046002568805
        }
      }
    }
  }
}

Getting token from http://192.168.39.88:30552/oauth/token
{"access_token":"83061770-7057-4e82-aa6d-ea310e9749b3","token_type":"bearer","expires_in":42543,"scope":"read write"}
RECEIVED RESPONSE:
meta {
  puid: "9li1oc1lr2ooc67g7hiuhmiuh3"
  requestPath {
    key: "complex-model"
    value: "model-with-feedback-grpc:0.1"
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  ndarray {
    values {
      list_value {
        values {
          number_value: 4.676817651188339

## Test feedback

In [40]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --endpoint send-feedback --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:
request {
  data {
    names: "sepal_length"
    names: "sepal_width"
    names: "petal_length"
    names: "petal_width"
    ndarray {
      values {
        list_value {
          values {
            number_value: 4.892674496650631
          }
          values {
            number_value: 4.742377470286767
          }
          values {
            number_value: 2.851798122512135
          }
          values {
            number_value: 0.004773596258239476
          }
        }
      }
    }
  }
}
response {
  data {
    names: "class1"
    names: "class2"
    names: "class3"
    ndarray {
      values {
        list_value {
          values {
            number_value: 0.8873412759534708
          }
          values {
            number_value: 0.6945005393322701
          }
          values {
            number_value: 0.18687912951259333
          }
        }
      }
    }
  }
}
reward: 1.0

Getting token from http://192.16

In [41]:
!kubectl delete -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


In [42]:
!minikube delete

Deleting local Kubernetes cluster...
Machine deleted.
